<a href="https://colab.research.google.com/github/vadManuel/Machine-Learning-UCF/blob/master/Homework/hw1/mvasquez_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

## Poblem 1
Find three simple functions from R^2 to R and suitable points that are:
- a local minimum
- a local maximum
- neither a minimum nor maximum, but the gradient is zero at this point.

Plot these functions using 3D plots as in the notebooks for creating density and contour plots and three dimensional plots. Indicate clearly the points. Use 2d and 3d contour plots.

In [0]:
def plot_util(func, x, y_range):
  return np.array([func(x, y) for y in np.linspace(y_range[0], y_range[1])])

def make_surface(func, x_range, y_range):
  x = np.linspace(x_range[0], x_range[1])
  z = plot_util(func, x[0], y_range)
  for i in range(1, len(x)):
    z = np.vstack((z, plot_util(func, x[i], y_range)))

  return go.Surface(z=z, showscale=False, colorscale='Viridis')

def make_line(func, x_range):
  x = np.linspace(x_range[0], x_range[1])
  y = np.vectorize(func)
  return go.Scatter(x=x, y=y(x_range, 0), mode='lines')

def multi_plot(plots, title=None):
  cols = 2
  rows = 1
  specs = [[{'type': 'surface'}]*cols]*rows
  fig = make_subplots(rows=rows, cols=cols, specs=specs)

  for row in range(rows):
    for col in range(cols):
      i = row*cols+col
      if i == len(plots): break
      fig.add_trace(plots[i], row=row+1, col=col+1)
  
  # fig.update_traces(contours_z=dict(show=True, color='rgb(150,150,150)', project_x=True, project_y=True, project_z=True))
  fig.update_layout(title=title, autosize=False,
                    margin=dict(l=10, r=0, b=10, t=40), height=400)

  fig.show()

In [0]:
def get_H(func_xx, func_yy, func_xy, x, y):
  return func_xx(x,y)*func_yy(x,y)-func_xy(x,y)**2

def get_critical_point_type(func_xx, func_yy, func_xy, x, y):
  if get_H(func_xx, func_yy, func_xy, x, y) > 0:  # H > 0 ::= max/min
    if func_xx(x, y) > 0: # func_xx(x, y) > 0 ::= min
      return 'min'
    if func_xx(x, y) < 0: # func_xx(x, y) < 0 ::= max
      return 'max'
    return 'unknown'      # func_xx(x, y) = 0 ::= unknown
  if get_H(func_xx, func_yy, func_xy, x, y) < 0:  # H < 0 ::= saddle point
    return 'saddle point'
  return 'unknown'                                # H = 0 ::= unknown

In [0]:
def f(x, y):
  return (x+3)**2+y**2
def f_not_z(x, _):
  return np.imag(1)*(x+3)
def f_x(x, y):
  return 2*(x+3)
def f_xx(x, y):
  return 2
def f_y(x, y):
  return 2*y
def f_yy(x, y):
  return 2
def f_xy(x, y):
  return 0

def g(x,y):
  return -x**2-y**2
def g_x(x,y):
  return -2*x
def g_xx(x,y):
  return -2
def g_y(x,y):
  return -2*y
def g_yy(x,y):
  return -2
def g_xy(x,y):
  return 0

def h(x,y):
  return x**2-y**2
def h_x(x,y):
  return 2*x
def h_xx(x,y):
  return 2
def h_y(x,y):
  return -2*y
def h_yy(x,y):
  return -2
def h_xy(x,y):
  return 0

def i(x,y):
  return x**3-y**3+(x*y)**2
def k(x,y):
  return (x-3)**2+(x+3)**2+y
def m(x,y):
  return np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)

In [0]:
plots = [make_surface(f, (-20,20), (-20,20)), make_surface(f_x, (-20,20), (-20,20)), make_surface(f_y, (-20,20), (-20,20)), make_surface(f, (-3,-3), (0,0)),
         make_surface(g, (-20,20), (-20,20)), make_surface(g_x, (-20,20), (-20,20)), make_surface(g_y, (-20,20), (-20,20)), make_surface(g, (0,0), (0,0)),
         make_surface(h, (-20,20), (-20,20)), make_surface(h_x, (-20,20), (-20,20)), make_surface(h_y, (-20,20), (-20,20)), make_surface(h, (0,0), (0,0))]

In [461]:
multi_plot(plots[:4], title='[P1] local minimum | f(x,y)=(x+3)^2+y^2')

In [190]:
multi_plot(plots[4:8], title='[P1] local maximum | f(x,y)=-x^2-y^2)')

In [191]:
multi_plot(plots[8:12], title='[P1] saddle point | f(x,y)=x^2-y^2')

In [0]:
def foo(func, x_range, y_range, critical_point=None, x_offset=0, y_offset=0, z_offset=-10):
  xx = np.linspace(x_range[0], x_range[1])
  yy = np.linspace(y_range[0], y_range[1])
  x,y = np.meshgrid(xx,yy)
  v_func = np.vectorize(func)
  z = v_func(x,y)

  trace = go.Surface(
      x=x,
      y=y,
      z=z,
      colorscale='rainbow',
      opacity=1,
      # showscale=False,
      name='')
  
  # x_offset = np.min(xx)*np.ones(z.shape)
  # y_offset = np.min(yy)*np.ones(z.shape)
  # z_offset = (np.min(z))*np.ones(z.shape)-10
  x_scale = abs(x_range[0])+abs(x_range[1])
  y_scale = abs(y_range[0])+abs(y_range[1])
  z_scale = abs(np.min([x,y,z]))+abs(np.max([x,y,z]))


  tracex = go.Surface(
      x=np.min(xx)*np.ones(z.shape)-x_scale*.15,
      y=y,
      z=z,
      colorscale='gray',
      opacity=.8,
      showscale=False,
      surfacecolor=z, # mapping depth
      name='yz-projection')
  tracey = go.Surface(
      x=x,
      y=np.min(yy)*np.ones(z.shape)-y_scale*.15,
      z=z,
      colorscale='gray',
      opacity=.8,
      showscale=False,
      surfacecolor=z, # mapping depth
      name='xz-projection')
  tracez = go.Surface(
      x=x,
      y=y,
      z=z_offset*np.ones(z.shape)-z_scale*.15,
      colorscale='gray',
      opacity=.8,
      showscale=False,
      surfacecolor=z, # mapping depth
      name='xy-projection')

  if critical_point:
    theta = np.linspace(0,2*np.pi)
    phi = np.linspace(0,np.pi)
    x = np.outer(np.cos(theta), np.sin(phi))*x_scale*.02  # reshape to 3D
    y = np.outer(np.sin(theta), np.sin(phi))*y_scale*.02  # reshape to 3D
    z = np.outer(np.ones(len(theta)), np.cos(phi))*z_scale*.015+z_scale*.5             # reshape to 3D

    point = go.Surface(
        x=x,y=y,z=z,
        # color='rgb(200,45,20)',
        showscale=False,
        # name='critical point'
    )
    return [trace, tracex, tracey, tracez, point]

  
  return [trace, tracex, tracey, tracez]
  # fig = go.Figure(data=data)
  # fig.show()

In [0]:
def foo2(plots, title=None):
  cols = 2
  specs = [[{'type': 'surface'}]*cols]
  fig = make_subplots(rows=1, cols=cols, specs=specs)

  for i in range(cols):
    for j in range(len(plots[i])):
      fig.add_trace(plots[i][j], row=1, col=i+1)
  
  fig.update_traces(contours_z=dict(show=True, color='rgb(205,105,105)', project_x=True, project_y=True, project_z=True))
  fig.update_layout(title=title, autosize=False,
                    margin=dict(l=10, r=0, b=10, t=40), height=400)

  fig.show()

In [555]:
a = foo(k, [-10,10], [-10,10], critical_point=[-3,0], x_offset=-2, y_offset=-2, z_offset=-2)
foo2([a,a])
# foo(i, [-2,1], [-2,1])

In [466]:
theta = np.linspace(0,2*np.pi)
phi = np.linspace(0,np.pi)
x = np.outer(np.cos(theta), np.sin(phi))  # reshape to 3D
y = np.outer(np.sin(theta), np.sin(phi))  # reshape to 3D
z = np.outer(np.ones(len(theta)), np.cos(phi))  # reshape to 3D
# go.Surface(z=z)

fig = go.Figure(data=[go.Surface(x=x,y=y,z=z)])
fig.update_layout(title='test')
fig.show()